In [0]:
import pandas as pd
import string
import nltk
import numpy as np
import sklearn
import seaborn as sns
import re
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import minmax_scale
from scipy import sparse
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report,accuracy_score

In [0]:
import torch 
from torchtext import data
import torch.nn as nn
import torchtext
import torch.nn.functional as F
from torch import optim

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Load Dataset**

In [0]:
train_data=pd.read_csv("/content/drive/My Drive/ML project'20/YelpZip/train_new.csv")
valid_data=pd.read_csv("/content/drive/My Drive/ML project'20/YelpZip/val_new.csv")
test_data=pd.read_csv("/content/drive/My Drive/ML project'20/YelpZip/test_new.csv")

In [6]:
train_data.head()

,user_id,product_id,rating,label,date,review_text,day,year,word_count,punctuation_count,char_count,title_count,user_id_no_of_review,user_id_ave_rating,user_id_ave_no_words,user_id_max_review_a_day,product_id_no_of_review,product_id_ave_rating,product_id_ave_no_of_words,product_id_max_review_a_day
0,207364,2043,5.0,1,2012-01-02,"ambiance: very casual, very trendy, very ... ...",Monday,2012,244,66,1376,30,1,5.000000,244.0,1,54,3.611111,133.240741,12
1,15736,5016,3.0,1,2012-05-29,the seafood was fresh and tasty. getting rese...,Tuesday,2012,44,6,231,5,6,4.333333,56.5,1,23,3.826087,90.347826,4
2,206677,1342,5.0,-1,2012-03-05,love the food and the amazing customer service...,Monday,2012,21,4,118,4,1,5.000000,21.0,1,5,4.200000,127.800000,2
3,174617,1526,5.0,1,2012-12-14,oh baby! roberta's has to be the coolest place...,Friday,2012,102,17,564,11,1,5.000000,102.0,1,69,3.840580,105.956522,9
4,23572,139,4.0,1,2012-06-30,"good food, good service, long waiting during t...",Saturday,2012,9,3,55,0,1,4.000000,9.0,1,61,4.114754,110.721311,14


In [0]:
pd.set_option('mode.chained_assignment', None)

In [0]:
Days={}
i=1
for val in train_data['day']:
  if val not in Days:
    Days[val]= i
    i+=1
train_data['day']=train_data['day'].apply(lambda x: Days[x])
test_data['day']=test_data['day'].apply(lambda x: Days[x])
valid_data['day']=valid_data['day'].apply(lambda x: Days[x])

In [0]:
train_data['label']=train_data['label'].apply(lambda x: 0 if x==-1 else 1)
test_data['label']=test_data['label'].apply(lambda x: 0 if x==-1 else 1)
Days={}
i=1
for val in train_data['day']:
  if val not in Days:
    Days[val]= i
    i+=1
train_data['day']=train_data['day'].apply(lambda x: Days[x])
test_data['day']=test_data['day'].apply(lambda x: Days[x])
valid_data['day']=valid_data['day'].apply(lambda x: Days[x])

In [0]:
tr_data=torch.zeros((len(train_data),13))
tst_data=torch.zeros((len(test_data),13))
val_data=torch.zeros((len(valid_data),13))
i=0
columns=list(train_data.drop(['user_id','product_id','rating','review_text','label','year','date'],axis=1).columns)
for col in columns:
  tr_data[:,i]=torch.tensor(train_data[col].to_numpy())
  i+=1
i=0
for col in columns:
  tst_data[:,i]=torch.tensor(test_data[col].to_numpy())
  i+=1
i=0
for col in columns:
  val_data[:,i]=torch.tensor(valid_data[col].to_numpy())
  i+=1

### **LSTM part**

In [0]:
#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [0]:
x=train_data[['review_text','label']].copy()
x.to_csv("custom_train.csv",index=False)
y=test_data[['review_text','label']].copy()
y.to_csv("custom_test.csv",index=False)
y=valid_data[['review_text','label']].copy()
y.to_csv("custom_valid.csv",index=False)

In [0]:
TEXT = torchtext.data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = torchtext.data.LabelField(dtype = torch.float,batch_first=True)
META = torchtext.data.RawField()

In [0]:
fields = [('user_id',None),('product_id',None),('rating',None), ('label', LABEL), ('date',None),\
          ('review_text',TEXT),('day',META),('year',None),('word_count',META),\
          ('punctuation_count',META),('char_count',META),('title_count',META),('user_id_no_of_review',META),\
          ('user_id_ave_rating',META),('user_id_ave_no_words',META),('user_id_max_review_a_day',META),\
          ('product_id_no_of_review',META),('product_id_ave_rating',META),('product_id_ave_no_of_words',META),\
          ('product_id_max_review_a_day',META)]
train__data=torchtext.data.TabularDataset(path = "/content/drive/My Drive/ML project'20/YelpZip/train_new.csv",format = 'csv',fields = fields,skip_header = True)
valid__data=torchtext.data.TabularDataset(path = "/content/drive/My Drive/ML project'20/YelpZip/val_new.csv",format = 'csv',fields = fields,skip_header = True)
test__data=torchtext.data.TabularDataset(path = "/content/drive/My Drive/ML project'20/YelpZip/test_new.csv",format = 'csv',fields = fields,skip_header = True)

In [0]:
print(vars(train__data.examples[0]))

{'label': '1', 'review_text': ['it', "'s", 'a', 'surprise', 'to', 'me', 'that', 'this', 'place', 'gets', 'such', 'stellar', 'reviews', '...', 'perhaps', 'that', 'is', 'merely', 'a', 'testament', 'to', 'the', 'general', 'state', 'of', 'ramen', 'depravity', 'in', 'prospect', 'heights', '.', 'the', 'pork', 'bun', 'appetizers', 'are', 'five', 'bucks', 'a', 'pop', '.', ' ', 'originally', 'i', 'was', 'thinking', '/', 'hoping', 'that', 'a', 'serving', 'would', 'come', 'with', '2', 'buns', '.', ' ', 'the', 'pork', 'bun', 'was', 'solid', 'but', 'nowhere', 'near', 'the', 'fatty', 'melt', '-', 'in', '-', 'your', '-', 'mouthiness', 'that', 'other', 'ramen', 'restaurants', '(', 'ahem', ',', 'ippudo', ')', 'deliver', '.', ' ', 'the', 'kale', 'salad', 'was', 'a', 'potent', 'combination', 'of', 'crunchiness', 'and', 'saltiness', 'and', 'by', 'far', 'my', 'favorite', 'part', 'of', 'the', 'meal', '.', 'as', 'for', 'the', 'ramen', '...', 'the', 'portions', 'are', 'definitely', 'on', 'the', 'smaller', 'si

In [0]:
# # fields = [ ('review_text',TEXT),('label', LABEL)]
# #loading custom dataset
# train__data=torchtext.data.TabularDataset(path = 'custom_train.csv',format = 'csv',fields = fields,skip_header = True)
# valid__data=torchtext.data.TabularDataset(path = 'custom_valid.csv',format = 'csv',fields = fields,skip_header = True)
# test__data=torchtext.data.TabularDataset(path = 'custom_test.csv',format = 'csv',fields = fields,skip_header = True)

In [15]:
for i in train__data:
  print(i)
  break

In [16]:
#initialize glove embeddings
TEXT.build_vocab(train__data,min_freq=2,vectors = "glove.6B.100d")
LABEL.build_vocab(train__data)

print("Size of TEXT vocabulary:",len(TEXT.vocab)) #No. of unique tokens in text
print("Size of LABEL vocabulary:",len(LABEL.vocab)) #No. of unique tokens in label
print(TEXT.vocab.freqs.most_common(10))  #Commonly used words
print(TEXT.vocab.stoi)  #Word dictionary

.vector_cache/glove.6B.zip: 862MB [06:29, 2.22MB/s]                          
 99%|█████████▉| 397892/400000 [00:18<00:00, 13218.52it/s]

Size of TEXT vocabulary: 23062
Size of LABEL vocabulary: 2
[('.', 181241), ('the', 161211), (',', 123648), ('and', 101379), ('i', 84661), ('a', 80023), ('to', 61905), ('was', 55755), ('it', 48369), (' ', 48304)]
defaultdict(<function _default_unk_index at 0x7efbc0e9e7b8>, {'<unk>': 0, '<pad>': 1, '.': 2, 'the': 3, ',': 4, 'and': 5, 'i': 6, 'a': 7, 'to': 8, 'was': 9, 'it': 10, ' ': 11, 'of': 12, 'is': 13, 'not': 14, 'for': 15, 'in': 16, '!': 17, 'with': 18, 'but': 19, 'that': 20, 'we': 21, 'you': 22, 'my': 23, 'this': 24, 'have': 25, 'on': 26, '-': 27, 'had': 28, 'they': 29, 'food': 30, 'were': 31, 'are': 32, 'good': 33, 'so': 34, 'place': 35, ')': 36, '(': 37, 'at': 38, 'as': 39, 'be': 40, 'great': 41, 'there': 42, "'s": 43, 'very': 44, 'all': 45, 'here': 46, 'like': 47, 'just': 48, 'if': 49, '"': 50, 'would': 51, 'out': 52, 'one': 53, 'do': 54, 'will': 55, 'which': 56, 'our': 57, 'really': 58, 'can': 59, 'get': 60, 'me': 61, 'service': 62, 'their': 63, '...': 64, 'go': 65, 'or': 66, '

In [17]:
TEXT.vocab.vectors.shape

torch.Size([23062, 100])

In [0]:
#check whether cuda is available
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 256

#Load an iterator
train_iterator,test_iterator,valid_iterator = torchtext.data.BucketIterator.splits(
    (train__data,test__data,valid__data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.review_text),
    sort_within_batch=True,
    device = device)

In [20]:
train_iterator

In [0]:
features=['user_id_no_of_review','user_id_ave_rating','user_id_ave_no_words','user_id_max_review_a_day',\
          'product_id_no_of_review','product_id_ave_rating','product_id_ave_no_of_words','product_id_max_review_a_day',\
          'word_count','punctuation_count','char_count','title_count','day']

# **LSTM Model**

In [0]:
# Nerural network module of torch
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)

        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs


In [0]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.5

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [42]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(23062, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 2,365,657 trainable parameters
torch.Size([23062, 100])


In [0]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [0]:
# train function for Bi-LSTM
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    predicted=[]
    truth=[]
    #set the model in training phase
    model.train()  
    i=0
    f1=0
    for batch in iterator:
        # if(i%10==0):
        #   print(i)
        # i+=1
        #resets the gradients after every batch
        optimizer.zero_grad()
        
        #convert to 1D tensor
        predictions = model(text, text_lengths)
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropagate the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        predicted += list(predictions.detach().cpu().numpy().round())
        truth += list(batch.label.detach().cpu().numpy())
        
    f1 = f1_score(truth, predicted)
    return epoch_loss / len(iterator), epoch_acc / len(iterator),f1

In [0]:
from sklearn.metrics import f1_score
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0
    predicted=[]
    truth=[]
    model.eval()
    f1=0
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.review_text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths)
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()

            predicted += list(predictions.detach().cpu().numpy().round())
            truth += list(batch.label.detach().cpu().numpy())
            
            
    f1 = f1_score(truth, predicted)
    report=classification_report
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1,report

In [74]:
N_EPOCHS = 3
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc,f1_train = train(model, train_iterator, optimizer, criterion)
    
    valid_loss,valid_train,f1_valid,Cl_report= evaluate(model,valid_iterator,optimizer,criterion)

    #evaluate the model
    test_loss, test_acc,f1_test,Cl_report = evaluate(model, test_iterator, criterion)

    #save the best model
    if test_loss < best_valid_loss:
        best_valid_loss = test_loss
        torch.save(model.state_dict(), 'saved_weights_cnn.pt')
    print("For Epoch {}".format(epoch+1))
    print(f'\tTrain Loss: {train_loss:.3f}   | Train Acc: {train_acc*100:.2f}%')
    print(f'\tTest  Loss: {test_loss:.3f}   | Test Acc: {test_acc*100:.2f}%')
    print(f'\tValid  Loss: {valid_loss:.3f}   | Valid Acc: {valid_acc*100:.2f}%')
    print(f'\tF1-Score train: {f1_train:.4f} | F1-Score test: {f1_test:.4f}   | F1-Score valid: {f1_valid: .4f}')

For Epoch 1
	Train Loss: 0.672   | Train Acc: 72.52%
	Test  Loss: 0.708   | Test Acc : 67.96%
	Valid Loss: 0.691   | Valid Acc: 67.95%
	F1 train  : 0.6667  | F1 Valid : 0.6514 | F1 Test : 0.6525
For Epoch 2
	Train Loss: 0.580   | Train Acc: 75.52%
	Test  Loss: 0.665   | Test Acc : 69.65%
	Valid Loss: 0.654   | Valid Acc: 68.20%
	F1 train  : 0.6841  | F1 Valid : 0.6567 | F1 Test : 0.6700
For Epoch 3
	Train Loss: 0.550   | Train Acc: 76.90%
	Test  Loss: 0.618   | Test Acc : 69.82%
	Valid Loss: 0.603   | Valid Acc: 70.18%
	F1 train  : 0.6912  | F1 Valid : 0.6600 | F1 Test : 0.6712
